In [1]:
#preprocessing dataset
#removing nans with mean value

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df = pd.read_csv('Train.csv')
df2 = pd.read_csv('Test.csv')
X_train = np.zeros((8523,11))
X_test = np.zeros((5681,11))
list1 = ['Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']
list2 = ['Item_Weight' , 'Item_Visibility', 'Item_MRP']
cnt = 0 
for item in list1: 
    df[item] = df[item].replace(np.nan, '', regex=True)
    temp = df[item].values
    le.fit(temp)
    temp = le.transform(temp) 
    mean_value = np.mean(temp[temp > 0])
    temp[temp == 0] = mean_value
    X_train[:,cnt] = temp
    df2[item] = df2[item].replace(np.nan, '', regex=True)
    temp2 = df2[item].values
    le.fit(temp2)
    temp2 = le.transform(temp2) 
    mean_value2 = np.mean(temp2[temp2 > 0])
    temp2[temp2 == 0] = mean_value2
    X_test[:,cnt] = temp2
    cnt = cnt+1
for item in list2:
    df[item] = df[item].replace(np.nan, 0, regex=True)
    temp = df[item].values
    mean_value = np.mean(temp[temp > 0])
    temp[temp == 0] = mean_value
    X_train[:,cnt] = temp
    df2[item] = df2[item].replace(np.nan, 0, regex=True)
    temp2 = df2[item].values
    mean_value2 = np.mean(temp2[temp2 > 0])
    temp2[temp2 == 0] = mean_value2
    X_test[:,cnt] = temp2
    cnt = cnt+1
Y_train = np.zeros((8523,1))
Y_test = np.zeros((8523,1))
df['Item_Outlet_Sales'] = df['Item_Outlet_Sales'].replace(np.nan, 0, regex=True)
temp = df['Item_Outlet_Sales'].values
mean_value = np.mean(temp[temp > 0])
temp[temp == 0] = mean_value
Y_train[:,0] = temp

In [2]:
W1 = tf.get_variable("W1", [10,11], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
W2 = tf.get_variable("W2", [5,10], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
W3 = tf.get_variable("W3", [1,5], initializer = tf.contrib.layers.xavier_initializer(seed = 1))
b1 = tf.get_variable("b1", [1,10], initializer = tf.zeros_initializer())
b2 = tf.get_variable("b2", [1,5], initializer = tf.zeros_initializer())
b3 = tf.get_variable("b3", [1,1], initializer = tf.zeros_initializer())
parameters = {"W1":W1 , "W2":W2,"W3":W3, "b1":b1, "b2":b2, "b3":b3 }


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
def forward_propagation(X, parameters):
    W1 = parameters['W1']
    W2 = parameters['W2']
    W3 = parameters['W3']
    b1 = parameters['b1']
    b2 = parameters['b2']
    b3 = parameters['b3']
    Z1 = tf.add(tf.matmul(X,tf.transpose(W1)),b1)
    A1 = tf.nn.relu(Z1)
    Z2 = tf.add(tf.matmul(A1,tf.transpose(W2)),b2)
    A2 = tf.nn.relu(Z2)
    Z3 = tf.add(tf.matmul(A2,tf.transpose(W3)),b3)
    return Z3

In [9]:
XP = tf.placeholder(tf.float32,shape=(8523,11), name="X")
XP1 = tf.placeholder(tf.float32,shape=(5681,11), name="X1")
YP = tf.placeholder(tf.float32,shape=(8523,1), name="Y")
Z3 = forward_propagation(XP,parameters)
print(Y_train.shape)
print(Z3.shape)
cost = tf.sqrt(tf.reduce_mean(tf.losses.mean_squared_error(Y_train,Z3))/8523)
optimizer = tf.train.AdamOptimizer(learning_rate = 0.003).minimize(cost)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(10000):
    _,min_cost = sess.run([optimizer,cost] ,feed_dict={XP : X_train, YP: Y_train})

(8523, 1)
(8523, 1)


In [18]:
Z4 = forward_propagation(XP1,parameters)
Z4 = sess.run(Z4,feed_dict={XP1 : X_test} )
tem1 = df2['Item_Identifier'].values
tem2 = df2['Outlet_Identifier'].values
mycsv = open("output.csv", "w")
columnTitleRow = "Item_Identifier,Outlet_Identifier,Item_Outlet_Sales\n"
print(tem2.shape)
mycsv.write(columnTitleRow)
for i in range(5681):
    row = str(tem1[i]) + "," + str(tem2[i]) + "," + str(Z4[i][0]) + "\n"
    mycsv.write(row)

(5681,)
